# RSS reader

## References

- https://www.w3schools.com/xml/xml_rss.asp
- https://towardsdatascience.com/how-i-turned-my-companys-docs-into-a-searchable-database-with-openai-4f2d34bd8736

## Dependencies & config

In [2]:
# dependencies
%pip install feedparser openapi python-dotenv beautifulsoup4 numpy qdrant-client uuid

Note: you may need to restart the kernel to use updated packages.


In [3]:
# config
from dotenv import load_dotenv
import openai
import os
import numpy as np

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## RSS feeds

First, we need to import the RSS feeds.
There is a top level `channel`, with:

- `title`
- `link`
- `description`
- `category`: optional
- `copyright`: optional
- `image`: optional, has `url`, `title`, `link`
- `language`: optional
- `pubDate`: optional
- a list of `items`

Each item has:

- `title`
- `link`
- `description`
- `author`: optional
- `comments`: optional
- `pubDate`: optional
- `enclosure`: optional, `url`, `length`, `type` attributes
- `content.encoded`: optional, wrapped in `![CDATA[` + `]]`

In [4]:
import feedparser
import pprint

def extract_feed(url: str):
    """
    Extracts the feed from the given URL
    """
    parsed_feed = feedparser.parse(f)
    feed = {}
    
    feed['title'] = parsed_feed.feed.get('title', None)
    feed['description'] = parsed_feed.feed.get('description', None)
    feed['link'] = parsed_feed.feed.get('link', None)
    if 'image' in parsed_feed.feed:
        feed['image_title'] = parsed_feed.feed.image.get('title', None)
        feed['image_link'] = parsed_feed.feed.image.get('link', None)
    feed['items'] = []

    for e in parsed_feed.entries:
        item = {}
        item['title'] = e.get('title', None)
        item['link'] = e.get('link', None)
        item['description'] = e.get('description', None)
        item['pubDate'] = e.get('pubDate', None)
        item['comments'] = e.get('comments', None)
        item['content_encoded'] = e.get('content.encoded', None)
        if 'image' in e:
            item['image_title'] = e.image.get('title', None)
            item['image_link'] = e.image.get('link', None)
        feed['items'].append(item)

    return feed    

feeds_list = ['https://news.ycombinator.com/rss']
# , 'https://themacrocompass.substack.com/feed']
feeds = []
for f in feeds_list:
    feed = extract_feed(f)
    pprint.pprint(feed, indent=4)
    feeds.append(feed)

{   'description': 'Links for the intellectually curious, ranked by readers.',
    'items': [   {   'comments': 'https://news.ycombinator.com/item?id=36491514',
                     'content_encoded': None,
                     'description': '<a '
                                    'href="https://news.ycombinator.com/item?id=36491514">Comments</a>',
                     'link': 'https://neal.fun/deep-sea/',
                     'pubDate': None,
                     'title': 'The Deep Sea (2019)'},
                 {   'comments': 'https://news.ycombinator.com/item?id=36491704',
                     'content_encoded': None,
                     'description': '<a '
                                    'href="https://news.ycombinator.com/item?id=36491704">Comments</a>',
                     'link': 'https://saurabhs.org/advanced-macos-commands',
                     'pubDate': None,
                     'title': 'macOS command-line tools you might not know '
                            

We extract the text content with `BeautifulSoup` and `lxml`.

In [24]:
from bs4 import BeautifulSoup
import requests
import pprint

def extract_feed_text(feed: dict):
    """
    Extract the text content of a link
    """
    for item in feed['items']:
        link = item['link']
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(strip = True)
        text = text.replace('\n', ' ')
        item['text'] = text
        print('---')
        print(item['title'])
        print(item['text'])

for feed in feeds:
    print(feed['title'])
    extract_feed_text(feed)
    # pprint.pprint(feed, indent=4)

Hacker News
---
The Deep Sea (2019)
The Deep SeaThe Deep SeaMade withby Neal AgarwalManateeBottlenose Dolphin DiveGreen Sea TurtleBeluga WhaleSea LionVelvet CrabStaghorn CoralKiller WhaleBarramundiGreat BarracudaSpotted BassStriped BassBlack DrumBlue FishSpiny dogfishDentexMahi-mahiFlounderBull SharkGreat White SharkBlue SharkGummy SharkMako SharkSunfishHumanAtlantic MackerelQueen SnapperPelagic StingrayDeepest dive of a NarwhalFrilled SharkViperfishAnglerfishLeatherback Sea TurtleOlive Ridly Sea TurtleSea PenDragonfishOrange RoughyWolf EelSwordfishChain CatsharkAtlantic CodPacific CodEuropean pilchardAtlantic SalmonChinook SalmonBlue TangClown FishHaddockVampire SquidJapanese Spider CrabFirefly SquidSperm Whale DiveYeti CrabBig Red JellyfishJewel SquidCockatoo SquidPhronimaBubblegum CoralGiant IsopodCoelacanthColossal SquidGoblin SharkChimaerasBlack SwallowerMonkfishGiant Pacific OctopusSixgill SharkEmperor Penguin DiveElephant Seal DiveBaird's Beaked WhaleLeptoserisGigantactisBigeye 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


---
Modern TLS/SSL on 16-bit Windows
Modern TLS/SSL on 16-bit WindowsDialup.netWindleWinGPTModern TLS/SSL on 16-bit WindowsLately, there's been an resurgence of new programs written for retro computers—everything     from a Slack client to many Wordle clones, to a Mastodon client. But most of these programs, if they     connect to the Internet, require a proxy running on a modern computer to handle     the SSL/TLS connection, which almost all APIs nowadays require. For my Gateway 4DX2-66 running     Windows 3.11 for Workgroups, making it reliant on a modern machine for any kind of real Internet     use is a sad state of affairs—so I decided to change the status quo.It wasn't that Windows 3.1 didn't support secure connections; Internet Explorer 2, for instance,     supported SSL. But over time, both clients and servers have upgraded to newer versions of the SSL     (now called TLS) protocol and algorithms, and have dropped support for older versions as     vulnerabilities likePOODLEare 

## Summarize

We use the OpenAI API to summarize the relevant documents.


In [40]:
OPENAI_API_MODEL = 'gpt-3.5-turbo'

def summarize(url:str):
  """
  Summarize the feed link
  """
  completion = openai.ChatCompletion.create(
    model=OPENAI_API_MODEL,
    messages=[
      {'role': 'system', 'content': "You are an assistant which reads and summarizes articles."},
      {'role': 'user', 'content': f"Summarize this: {url}"}
    ]
  )   
  return completion.choices[0].message.content

for i, feed in enumerate(feeds):
    print("#articles:", feed['items'].__len__())
    for j, item in enumerate(feed['items']):
      url = item['link']
      summary = summarize(url)
      item['summary'] = summary
      print(f"article #{j}")
      print(url)
      print(summary)

#articles: 30
article #0
https://neal.fun/deep-sea/
The website "The Deep Sea" created by neal.fun is an interactive infographic that explores the depths of the ocean and the creatures that live within it. It provides a vertical view of the ocean in relation to how deep different creatures live. The infographic highlights various sea creatures ranging from the smallest fish to the largest known animal on earth, the blue whale. The website allows you to click on each creature to learn more about their characteristics and habits. Additionally, the website shows the impacts of human activities on marine life through trash and pollution. Overall, the website offers a unique and informative look at the wonders and challenges of the underwater world.
article #1
https://saurabhs.org/advanced-macos-commands
The article provides a list of advanced Mac OS commands that can be used in the Terminal application. The commands range from navigating the file system to managing permissions and network 

## Embeddings

We determine the embeddings. OpenAI recommends `text-embedding-ada-002 model`, which is cheaper, faster, etc ... This gives embeddings of dimensions 1536. We need to create an embeddings for each text block. `text-embedding-ada-002` has a max input token of 8191.

In [41]:
OPENAI_EMBEDDINGS_MODEL = "text-embedding-ada-002"

def get_embdeddings(text: str):
    """
    Get the OpenAI embeddings of the text
    """
    response = openai.Embedding.create(
            input=text,
            model=OPENAI_EMBEDDINGS_MODEL
        )
    embeddings = response['data'][0]['embedding']
    return embeddings

for i, feed in enumerate(feeds):
    for j, item in enumerate(feed['items']):
        summary = item['summary']
        embeddings = get_embdeddings(summary)

## Vector DB

We store all embeddings inside a vector DB.

In [ ]:
import qdrant_client as qc
import qdrant_client.http.models as qmodels
import uuid

client = qc.QdrantClient(url="localhost")
METRIC = qmodels.Distance.COSINE
DIMENSION = 1536
COLLECTION_NAME = "dev_feeds"

def create_index():
    """
    Create index with default parameters
    """
    client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config = qmodels.VectorParams(
            size=DIMENSION,
            distance=METRIC,
        )
    )

def create_subsection_vector(
    subsection_content,
    section_anchor,
    page_url,
    doc_type
    ):

    vector = embed_text(subsection_content)
    id = str(uuid.uuid1().int)[:32]
    payload = {
        "text": subsection_content,
        "url": page_url,
        "section_anchor": section_anchor,
        "doc_type": doc_type,
        "block_type": block_type
    }
    return id, vector, payload

In [ ]:
# Add vectors to collection
def add_doc_to_index(subsections, page_url, doc_type, block_type):
    ids = []
    vectors = []
    payloads = []
    
    for section_anchor, section_content in subsections.items():
        for subsection in section_content:
            id, vector, payload = create_subsection_vector(
                subsection,
                section_anchor,
                page_url,
                doc_type,
                block_type
            )
            ids.append(id)
            vectors.append(vector)
            payloads.append(payload)
    
    ## Add vectors to collection
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=qmodels.Batch(
            ids = ids,
            vectors=vectors,
            payloads=payloads
        ),
    )

## Search space

To create a search on the index, we need to get the embeddings for the query string, and search the vector DB for the closest embeddings.

In [ ]:
def search_idx(query: str):
    """
    Generates a 
    """
    response = client.search(
        collection_name=COLLECTION_NAME,
        vector=embed_text(query),
        filter=None,
        top=5,
        params=qmodels.SearchRequestParams(
            hnsw_ef=128,
            hnsw_ef_search=128,
            is_reversed_index=True,
            is_async=False,
            timeout=1000,
        ),
    )
    return response